In [62]:
# imports
import pandas as pd
import numpy as np
# import seaborn as sns

In [63]:
# load data
df = pd.read_csv('./data/air_quality.csv')
df['code'] = df['code'].astype('category')
df['owner'] = df['owner'].astype('category')
df['name'] = df['name'].astype('category')
df['actualized'] = pd.to_datetime(df['actualized'])
df['owner'] = df['owner'].cat.rename_categories({'ZÚ-Ostrava':'ZU-Ostrava', 'ČHMÚ': 'CHMU'})
df['name'] = df['name'].cat.rename_categories({
    'Brno - Dětská nemocnice': 'Brno - Detska nemocnice',
    'Brno-Komárov': 'Brno-Komarov',
    'Brno-Lány': "Brno-Lany", 
    'Brno-Líšeň': 'Brno-Lisen', 
    'Brno-Masná': "Brno-Masna", 
    'Brno-Tuřany': 'Brno-Turany', 
    'Brno-Výstaviště': 'Brno-Vystaviste', 
    'Brno-Zvonařka': 'Brno-Zvonarka',
    'Brno-Úvoz (hot spot)': 'Brno-Uvoz (hot spot)'
})

In [64]:
# transform the table
df_transform = df.copy(deep=True)
measurements = ['so2_1h', 'no2_1h', 'co_8h', 'pm10_1h', 'o3_1h', 'pm10_24h', 'pm2_5_1h']
header = [i for i in df.columns if i not in measurements]
df_transform = (df_transform
    .melt(id_vars=header, value_vars=measurements, value_name='value')
    .dropna()
    .reset_index()
    .rename(columns={'variable': 'type'})
    .drop(columns='index')
)
df_transform = df_transform[['type', 'value'] + header]
df_transform

,type,value,X,Y,objectid,code,name,owner,lat,lon,actualized,globalid
0,so2_1h,6.7,16.696217,49.148972,95047,BBNYA,Brno-Turany,CHMU,49.148972,16.696217,2022-10-11 10:29:55+00:00,{C25A452E-5A5C-4776-83D1-8F5AA39F7673}
1,so2_1h,6.9,16.580812,49.165260,95050,BBMLA,Brno-Lany,SMBrno,49.165260,16.580812,2022-10-11 10:29:55+00:00,{E308C3B1-FFB7-40B4-81DA-B7A3A4C65D6A}
2,so2_1h,6.4,16.696217,49.148972,95058,BBNYA,Brno-Turany,CHMU,49.148972,16.696217,2022-10-11 11:29:37+00:00,{B7592BF8-F35F-4F2A-8478-5EA250E488DA}
3,so2_1h,8.8,16.580812,49.165260,95062,BBMLA,Brno-Lany,SMBrno,49.165260,16.580812,2022-10-11 11:29:37+00:00,{24466715-8BB7-4290-BDA5-0A4AB9E4CFD7}
4,so2_1h,8.8,16.580812,49.165260,95064,BBMLA,Brno-Lany,SMBrno,49.165260,16.580812,2022-10-11 13:29:38+00:00,{25091D1B-85B1-475A-82A7-1BD15B3CB2A5}
...,...,...,...,...,...,...,...,...,...,...,...,...
215717,pm2_5_1h,4.0,16.642517,49.208160,145364,BBMSA,Brno-Svatoplukova,SMBrno,49.208160,16.642517,2023-10-07 13:56:27+00:00,{BD257CC5-62E1-440E-8559-34A4DDF2F577}
215718,pm2_5_1h,7.6,16.616287,49.202724,145365,BBDNA,Brno - Detska nemocnice,CHMU,49.202724,16.616287,2023-10-07 13:56:27+00:00,{B8213486-4FB5-4822-BD72-7C505EE35374}
215719,pm2_5_1h,4.8,16.613836,49.216087,145366,BBMAA,Brno-Arboretum,SMBrno,49.216087,16.613836,2023-10-07 13:56:27+00:00,{28E65455-5AA6-4C8E-BB44-171FB6010F6D}
215720,pm2_5_1h,6.6,16.568829,49.189777,145367,BBMVA,Brno-Vystaviste,SMBrno,49.189777,16.568829,2023-10-07 13:56:27+00:00,{80F59240-BF4E-470B-8637-7147723BDFBA}


In [1]:
# create a txt file for influx v1
df_new = df_transform.copy(deep=True)
measurements = ['type']
tags = ['code', 'owner']
values = ['value', 'X', 'Y', 'name']
time = ['actualized']
df_new = df_new[measurements + tags + values + time]
df_new['type'] = df_new['type'].astype('str')
df_new['code'] = 'code=' + df_new['code'].astype('str')
df_new['owner'] = 'owner=' + df_new['owner'].astype('str')
df_new['value'] = 'value=' + df_new['value'].astype('str')
df_new['name'] = 'name="' + df_new['name'].astype('str') + '"'
df_new['X'] = 'X=' + df_new['X'].astype('str')
df_new['Y'] = 'Y=' + df_new['Y'].astype('str')
# df_new['actualized'] = df_new['actualized'].df.values
df_new['actualized'] = df_new['actualized'].astype('int64') // 10**9
_t = df_new.to_numpy()
# np.savetxt('./data/air_quality_formatted_v1.txt', _t, delimiter='', fmt='%s,%s,%s %s,%s,%s,%s %d')


NameError: name 'df_transform' is not defined

In [67]:
# df_transform.to_csv('./data/air_quality_formatted.csv', index=False)